## FAQ answering system

##### Author: [Alejandro Salmerón](https://www.linkedin.com/in/alejandrosalme/)

##### PREAMBLE

FAQs (Frequently Asked Questions) serve as a critical component of information dissemination for businesses and organizations. They play a pivotal role in enhancing customer experience, streamlining communication, and providing quick access to essential information. 

Using a generative QA system we can generate answers to questions in a free-form manner, similar to how a human might produce a response. While Non-Generative QA systems rely on pre-defined answers stored in a knowledge base. Based on that information, the importance and the benefits that this type of system could produce to a company, we will inspect an initial aproach of how to implement and use them with Haystack. Which is an end-to-end NLP framework that enables you to build applications powered by LLMs, Transformer models, vector search and more. 

In [ ]:
import pandas as pd
import requests
import textwrap
import re
from bs4 import BeautifulSoup

from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.pipelines import FAQPipeline, Pipeline, DocumentSearchPipeline
from haystack.utils import  add_example_data, print_documents
from haystack.nodes import Docs2Answers
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from text2speech import AnswerToSpeech
from getpass import getpass

from IPython.display import display, Audio
import soundfile as sf
from pathlib import Path

First we start scraping FAQs from HBO, Aliexpress and Whatsapp.

- Get, clean and store data

In [29]:
hbo='https://www.hbo.com/about/faqs'
aliexpress='https://sell.aliexpress.com/__pc/faq.htm'
whatsapp='https://business.whatsapp.com/resources/faq?lang=en_US'

urls = [hbo, aliexpress, whatsapp]

for url in urls:
    #Get
    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, 'html.parser') 
        elements = soup.find_all(['h3', 'h4','p'])

        print('FAQs:\n', elements[0:4])
        
        raw_faq_data = '\n'.join([p.get_text() for p in elements])
        " ".join(raw_faq_data.split())

        #Clean
        patterns=[(r'\s+\n','\n'), (r'Q: ', ''), (r'A: ', ''), (r'- ', '') ]
        for pattern, replacement in patterns:
            raw_faq_data = re.sub(pattern, replacement, raw_faq_data) 

        faq_from=url.split('.com')[0].split('.')[1]

        #Store
        with open('./data/faq_'+faq_from+'_data.txt', encoding='utf-8', mode='w') as f:
            f.writelines(raw_faq_data)

    else:
        print(f'Error fetching URL. Status code: {response.status_code}')

FAQs:
 [<h3 class="modal-title">max-style</h3>, <p> </p>, <h4 class="text-left"><strong>What does HBO stand for?</strong></h4>, <p>HBO stands for Home Box Office. This name reflects HBO’s origins, which emphasized airing uncut, ad-free movies. Today, HBO is known for providing a wide array of home entertainment, from blockbuster <a href="https://www.hbo.com/movies">movies</a> to award-winning <a href="https://www.hbo.com/series">series</a>.</p>]
FAQs:
 [<p class="title3">- Why sell on AliExpress?</p>, <p>AliExpress is the global platform of the Alibaba Group, present in more than 190 countries and regions. More than 150 million customers trust AliExpress sellers.</p>, <p><a></a></p>, <p class="title3">- In what categories of AliExpress can I sell?</p>]
FAQs:
 [<h3 class="_9vd5 _9sc-" style="color:#526571;">Products</h3>, <h3 class="_9vd5 _9sc-" style="color:#526571;">Resources</h3>, <h3 class="_9vd5 _9sc-" style="color:#526571;">Business Platform</h3>, <h3 class="_9vd5 _9sc- _9sd3" sty

### Non-Generative FAQ answering system

- Creating an in-memory database (DocumentStore) that stores the Documents that the QA system uses to find answers to our questions.

    InMemoryDocumentStore is the simplest DocumentStore. But for a faster and scalable text-focused storage option we could use an ElasticsearchDocumentStore which connects to a running Elasticsearch service that persists even after the Haystack program has finished running. 

In [3]:
document_store = InMemoryDocumentStore(use_bm25=True)

add_example_data(document_store, "./faq_store")

Preprocessing: 0docs [00:00, ?docs/s]


- Definining Retriever with a sentence similarity model (all-MiniLM-L6-v2), so we can find similar questions between the user's and the FAQ.

In [ ]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    use_gpu=True,
    scale_score=False,
)

- Parse data to get questions and answers

    We are merging all the FAQs to test if it's possible to ask questions about different products while maintaining meaningful answers.

In [5]:
faq_from=['hbo', 'aliexpress', 'whatsapp']
raw_faq_data = []
for f_from in faq_from:
    with open('./data/faq_'+ f_from +'_data.txt', 'r') as f:
        raw_faq_data+=f.readlines()
        raw_faq_data = [line for line in raw_faq_data if line.strip()] 

question = ''
answer = ''
qa_tuple=[]

for line in raw_faq_data:
    line = line.replace('\n', '')
    line = line.replace(u'\u200b', '')
    
    if '?' in line:
        if (question!='' and answer!=''):
            qa_tuple.append(tuple((question, answer)))
        
        question = line
        answer = ''
    else:
        answer = answer + line

- Creating embeddings for our questions as we want to match incoming question with stored question.

In [6]:
df_qa = pd.DataFrame(qa_tuple, columns=['content', 'answer'])

questions = list(df_qa["content"].values)
df_qa["embedding"] = retriever.embed_queries(queries=questions).tolist()

docs_to_index = df_qa.to_dict(orient="records")
document_store.write_documents(docs_to_index)

df_qa.head()

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Updating BM25 representation...: 100%|██████████| 36/36 [00:00<?, ? docs/s]


,content,answer,embedding
0,What does HBO stand for?,HBO stands for Home Box Office. This name refl...,"[0.018427811563014984, -0.010991335846483707, ..."
1,Who owns HBO?,"HBO is a subsidiary of Home Box Office, Inc., ...","[0.03248928114771843, -0.042040105909109116, -..."
2,What happened to HBO Max?,HBO Max is now Max. The enhanced service is st...,"[0.03972362354397774, -0.06855587661266327, -0..."
3,The last few minutes get cut off when I record...,"Sometimes, HBO programs may run for longer tha...","[0.04580259695649147, -0.04416085034608841, 0...."
4,Is HBO programming available for use in the cl...,Many HBO documentary titles are available for ...,"[0.03206067159771919, -0.023391274735331535, -..."


- Asking a different question (not stored in the data) and retrieve the top 5 similar questions 

In [7]:
pipeline_retrieval = DocumentSearchPipeline(retriever)
dict_res = pipeline_retrieval.run(query="Tell me something about WhatsApp Business Platform?", params={"Retriever": {"top_k": 5}})
print_documents(dict_res, max_text_len=512)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: Tell me something about WhatsApp Business Platform?

{'content': 'What is the WhatsApp Business Platform?', 'name': None}

{   'content': 'What are the options for integrating with the WhatsApp '
               'Business Platform?',
    'name': None}

{   'content': 'What are the options for technical support with the WhatsApp '
               'Business Platform?',
    'name': None}

{   'content': 'How is the WhatsApp Business Platform different from the '
               'Messenger Platform?',
    'name': None}

{   'content': 'How is the WhatsApp Business Platform different from the '
               'WhatsApp Business app?',
    'name': None}



- Pipeline initialization and answer inference

In [8]:
faq_pipeline = FAQPipeline(retriever=retriever)

In [9]:
question="How much is the WhatsApp Business platform?"

prediction = faq_pipeline.run(query=question, params={"Retriever": {"top_k": 1}})

print('Answer:\n', textwrap.fill(prediction["answers"][0].answer, width=100))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
 We charge based on the number of conversations. Charges will be per 24 hour conversation session,
with different rates for business-initiated and user-initiated conversations. A session initiates on
delivery of a business-initiated message, or delivery of a business reply to a user-initiated
message. A conversation session ends 24 hours after delivery of the first business message in the
conversation.The first 1,000 conversations each month are free so businesses can start to build
experiences their customers will love. When they’re ready to expand beyond 1,000 conversations per
month, our conversation pricing applies. Learn more and see our rate card.There is no additional fee
to access our platform directly. Partners may charge additional fees based on value-added services
or technologies they provide.


In [10]:
question="What is the cost of selling on AliExpress?"

prediction = faq_pipeline.run(query=question, params={"Retriever": {"top_k": 1}})

print('Answer:\n', textwrap.fill(prediction["answers"][0].answer, width=100))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:
 In AliExpress you will only pay for each product you sell. Commissions vary between 5% and 8%
depending on the product category.


- Another idea is to convert our answer into an audio file. We can do that only by adding a new node to our pipeline (AnswerToSpeech).

In [ ]:
a2s_pipeline = Pipeline()

answer2speech = AnswerToSpeech(
    model_name_or_path="espnet/kan-bayashi_ljspeech_vits", generated_audio_dir=Path("./audio_answers")
)

a2s_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
a2s_pipeline.add_node(component=Docs2Answers(), name="Docs2Answers", inputs=["Retriever"])
a2s_pipeline.add_node(answer2speech, name="AnswerToSpeech", inputs=["Docs2Answers"])

In [12]:
question="I have created a movie. How can I send it to HBO?"

prediction = a2s_pipeline.run(
    query=question, params={"Retriever": {"top_k": 1}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Converting answers to audio:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
print('Answer:\n', textwrap.fill(prediction["answers"][0].meta["answer"], width=100))

speech, _ = sf.read(prediction["answers"][0].answer)
display(Audio(speech, rate=24000))

Answer:
 We do not accept submissions directly — all submissions to HBO must come through an agent.
Therefore, we suggest that all scripts, screenplays, treatments, dramatizations, anecdotes,
treatises, rousing yarns, fables, inspiring narratives, thrilling legends, sagas, epic poetry,
intriguing whodunits, newspaper articles, one-act plays, short stories, skits, and novellas be sent
to the literary agency of your choice.


-------------

### Generative FAQ answering system + RAG (Retrieval-Augmentation)

- PromptNode gives the LLMs support needed. We can define de model and the prompt we want to be used, for a more human like and topic-focused response.

In [24]:
openai_api_key = getpass("Enter OpenAI API Key:")

rag_prompt = PromptTemplate(
    prompt="""Create an answer from the following text for the given question.
            Find an answer associated with the question and provide a clear and concise response.
            You are a chat assistant and your answer should be no longer than 100 words.
            \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(
    model_name_or_path="text-davinci-003", api_key=openai_api_key, default_prompt_template=rag_prompt
)

In [25]:
g_pipeline = Pipeline()
g_pipeline.add_node(component=retriever, name="retriever", inputs=["Query"])
g_pipeline.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

- Answers comparison between Generative and Non-Generative

   The Non-Generative system manages to obtain all the expected answers even though the questions change, this is due to the qa model and the similarity model used to find the most similar related question. While the Generative system provides more human and flexible answers with additional information that does not appear in the FAQ data used, which is knowledge provided by the model. One challenge of this type of models is to ensure that generated responses are always accurate. But by combining generative AI and retrieval like we did, the model uses FAQs documents as a kind of ground truth upon which to base its answers. 
   
   A deepest evaluation about the quality of results of the  question-answering pipeline could be made, but this is just and example and all anwsers seems to be correct.

In [27]:
questions = ["Does programs have a music playlist?", #HBO related
             "Where can I buy HBO merch?",
             "Are the WhatsApp Business Platform and app the same?", #Whatsapp related
             "Do I need to pay for selling?", #Aliexpress related
             "What I need to do to be a seller?"]

for q in questions:
    ng_prediction = faq_pipeline.run(query=q, params={"Retriever": {"top_k": 1}})
    g_prediction = g_pipeline.run(query=q)

    print('Question:', q, '\n')
    print('Non-GQ Answer:')
    print(textwrap.fill(ng_prediction["answers"][0].meta["answer"], width=100), '\n')
    print('GQ Answer:')
    print(textwrap.fill(g_prediction["answers"][0].answer, width=100),'\n')


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Does programs have a music playlist? 

Non-GQ Answer:
The music featured in programs is usually credited in closing credits, but sometimes producers may
withhold the information in anticipation of releasing a compilation soundtrack (as was done with the
first season of The Sopranos). Whenever possible, the episode information for a program will include
the music credits and track information along with a corresponding Apple Music link.Many promos use
excerpts from commercially-available music, but some are scored with original tunes and are
unavailable for download. 

GQ Answer:
Unfortunately, there is no official playlist of music used for HBO programs and promos. However,
music used in HBO programs and promos is typically either licensed or produced by HBO Music. To
obtain information on music used in HBO programs, you may contact the HBO Music department.
Additionally, HBO Licensing & Retail handles the development and distribution of soundtracks and
Home Video/DVDs. They 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Do I need to pay for selling? 

Non-GQ Answer:
In AliExpress you will only pay for each product you sell. Commissions vary between 5% and 8%
depending on the product category. 

GQ Answer:
Yes, you need to pay a certain commission for selling on AliExpress. To register as a seller, you
need to provide a valid identity document, a business license, and a valid bank account. You can
sell in various categories on AliExpress, such as Apparel, Electronics, Home & Garden, Beauty &
Health, and Toys & Hobbies. You can sell in any country where AliExpress has a presence. Some
categories of products are prohibited for selling on AliExpress, such as weapons, 



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: What I need to do to be a seller? 

Non-GQ Answer:
To register as a seller on AliExpress, you will be asked for a series of legal, fiscal and personal
documents depending on the legal nature of your company. At this time we only accept companies and
freelancers.Start signing up now at AliExpress 

GQ Answer:
In order to become a seller on AliExpress, you need to be registered with the AliExpress website.
You also need to have a valid business license, as well as a valid tax registration certificate. The
categories in which you can sell on AliExpress include Apparel & Accessories, Automobiles &
Motorcycles, Beauty & Health, Computer & Office, Home & Garden, Jewelry & Watches, Lights &
Lighting, Luggage & Bags, Mother & Kids, Shoes, Sports & Entertainment, 



### Conclusion

Considering that this is an small example, in this first aproach both systems seems to works well. So the choice between them depends on the specific requirements of the application, the availability of data, and the desired level of control over the answers provided. If we want diverse responses the Generative system is an option, but if accuracy needs to be ensured and we can  provide answers from a predefined set, the Non-Generative QA maybe the best option.

In some cases, a hybrid approach that combines elements of both approaches may also be effective.